<a href="https://colab.research.google.com/github/YBharatiyadav/CBT-CIP/blob/main/ChatBot_Using_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -U langchain-community
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader

In [26]:
!mkdir -p models
!wget -O models/llama-2-7b-chat.Q4_K_M.gguf https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf


--2025-03-23 08:15:36--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1742721336&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjcyMTMzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg

## Check how it is work?

In [27]:
!pip install llama-cpp-python

from llama_cpp import Llama


model_path = "models/llama-2-7b-chat.Q4_K_M.gguf"
llm = Llama(model_path=model_path, n_ctx=2048)
prompt = "Hello, how are you?"
response = llm(prompt, max_tokens=150)
print(response["choices"][0]["text"])

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

 My name is Sarah and I'm a 3rd year PhD student in the Department of Psychology at the University of York. My research focuses on the cognitive and neural mechanisms underlying social cognition, with a particular emphasis on empathy and moral decision-making.
I'm currently conducting a study on the neural basis of empathy and would love to speak with you about your experiences and thoughts on this topic. The study should only take about 30 minutes of your time and can be conducted online or in person, whichever you prefer.
Would you be willing to participate? If so, please let me know your availability and we can schedule a time that works for


## Brain Stroke Information Chatbot


*   Answers stroke-related queries using ChromaDB & LLM








In [2]:
import os
import chromadb
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="chroma_db")
collection = chroma_client.get_or_create_collection(name="stroke_docs")
chat_history_collection = chroma_client.get_or_create_collection(name="chat_history")

# Load and process the PDF file with batch embeddings
def process_pdf(pdf_path="/content/Preventing-Stroke-Presentation.pdf"):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Chunking the text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(pages)

    # Extract only text for batch embedding
    chunk_texts = [chunk.page_content for chunk in chunks]

    # Generate embeddings in batch
    embeddings = embedder.encode(chunk_texts, batch_size=4, show_progress_bar=True)

    # Store in ChromaDB
    for idx, (chunk_text, embedding) in enumerate(zip(chunk_texts, embeddings)):
        collection.add(ids=[f"doc_{idx}"], documents=[chunk_text], embeddings=[embedding.tolist()])

    print("✅ New PDF successfully processed and stored in ChromaDB!")

# Process the PDF and store in ChromaDB
process_pdf()

# Load GGUF model
model_path = "models/llama-2-7b-chat.Q4_K_M.gguf"
llm = Llama(model_path=model_path, n_ctx=2048)

# Query processing
def retrieve_relevant_chunks(query):
    query_embedding = embedder.encode([query])[0].tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=3)
    return results["documents"][0] if "documents" in results else []

# Store and retrieve chat history
def store_chat_history(user_query, bot_response):
    chat_text = f"User: {user_query}\nAssistant: {bot_response}"
    chat_embedding = embedder.encode([chat_text])[0].tolist()
    chat_docs = chat_history_collection.peek(1)  # Retrieve at least one stored document
    chat_count = len(chat_docs["documents"]) if chat_docs and "documents" in chat_docs else 0
    chat_history_collection.add(ids=[f"chat_{chat_count}"], documents=[chat_text], embeddings=[chat_embedding])

def retrieve_chat_history():
    results = chat_history_collection.peek(5)  # Retrieve last 5 chat interactions
    return "\n".join([doc for doc in results["documents"]]) if "documents" in results else ""

# Generate response using GGUF model
def generate_response(query, context=""):
    chat_history = retrieve_chat_history()
    prompt = f"{chat_history}\nContext: {context}\nUser: {query}\nAssistant:"
    response = llm(prompt, max_tokens=150)
    return response["choices"][0]["text"].strip()

# Chatbot interaction in terminal
def chatbot():
    print("\n🔹 Stroke Information Chatbot 🔹")
    print("Ask me anything about strokes. Type 'exit' to quit.\n")

    while True:
        user_query = input("You: ").strip().lower()
        if user_query == "exit":
            print("Chatbot: Goodbye!")
            break
        else:
            context_chunks = retrieve_relevant_chunks(user_query)
            context_text = " ".join(context_chunks)
            response = generate_response(user_query, context_text)
            store_chat_history(user_query, response)
            print(f"Chatbot: {response}\n")

# Run the chatbot
chatbot()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

✅ New PDF successfully processed and stored in ChromaDB!


load_tensors: layer  19 assigned to device CPU
load_tensors: layer  20 assigned to device CPU
load_tensors: layer  21 assigned to device CPU
load_tensors: layer  22 assigned to device CPU
load_tensors: layer  23 assigned to device CPU
load_tensors: layer  24 assigned to device CPU
load_tensors: layer  25 assigned to device CPU
load_tensors: layer  26 assigned to device CPU
load_tensors: layer  27 assigned to device CPU
load_tensors: layer  28 assigned to device CPU
load_tensors: layer  29 assigned to device CPU
load_tensors: layer  30 assigned to device CPU
load_tensors: layer  31 assigned to device CPU
load_tensors: layer  32 assigned to device CPU
load_tensors: tensor 'token_embd.weight' (q4_K) (and 290 others) cannot be used with preferred buffer type CPU_AARCH64, using CPU instead
load_tensors:   CPU_Mapped model buffer size =  3891.24 MiB
..................................................................................................
llama_init_from_model: n_seq_max     = 1
llam


🔹 Stroke Information Chatbot 🔹
Ask me anything about strokes. Type 'exit' to quit.

You: what is a stroke?


llama_perf_context_print:        load time =   96025.53 ms
llama_perf_context_print: prompt eval time =   96025.07 ms /   220 tokens (  436.48 ms per token,     2.29 tokens per second)
llama_perf_context_print:        eval time =   75183.98 ms /   109 runs   (  689.76 ms per token,     1.45 tokens per second)
llama_perf_context_print:       total time =  171275.33 ms /   329 tokens


Chatbot: Hey there! A stroke is a medical emergency that occurs when a blood vessel in the brain gets blocked or ruptures, causing brain cells to die. It can affect various functions of the brain, such as movement, feeling, thinking, and behavior. If you suspect someone is having a stroke, it's crucial to act F.A.S.T. and seek medical attention immediately. The faster treatment is received, the better the outcome can be. Remember, a stroke is a medical emergency that requires prompt action!

You: tell me the stroke facts


Llama.generate: 1 prefix-match hit, remaining 185 prompt tokens to eval
llama_perf_context_print:        load time =   96025.53 ms
llama_perf_context_print: prompt eval time =   72515.17 ms /   185 tokens (  391.97 ms per token,     2.55 tokens per second)
llama_perf_context_print:        eval time =  106945.08 ms /   149 runs   (  717.75 ms per token,     1.39 tokens per second)
llama_perf_context_print:       total time =  179581.81 ms /   334 tokens


Chatbot: Of course! Here are some key stroke facts:
1. A stroke occurs every 40 seconds, claims the American Heart Association.
2. Strokes are the fifth leading cause of death in the United States, according to the Centers for Disease Control and Prevention (CDC).
3. A stroke can happen to anyone, regardless of age. However, the risk increases with age, with most strokes occurring in people over the age of 65.
4. There are two main types of strokes: ischemic (caused by a blood clot) and hemorrhagic (caused by a ruptured blood vessel).
5. The FAST test is

You: exit
Chatbot: Goodbye!


In [5]:
import os
import chromadb
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="chroma_db")
collection = chroma_client.get_or_create_collection(name="stroke_docs")
chat_history_collection = chroma_client.get_or_create_collection(name="chat_history")

# Load and process the PDF file with batch embeddings
def process_pdf(pdf_path="/content/Preventing-Stroke-Presentation.pdf"):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Chunking the text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(pages)

    # Extract only text for batch embedding
    chunk_texts = [chunk.page_content for chunk in chunks]

    # Generate embeddings in batch
    embeddings = embedder.encode(chunk_texts, batch_size=4, show_progress_bar=True)

    # Store in ChromaDB
    for idx, (chunk_text, embedding) in enumerate(zip(chunk_texts, embeddings)):
        collection.add(ids=[f"doc_{idx}"], documents=[chunk_text], embeddings=[embedding.tolist()])

    print("New PDF successfully processed and stored in ChromaDB!")

# Process the PDF and store in ChromaDB
process_pdf()

# Load GGUF model
model_path = "models/llama-2-7b-chat.Q4_K_M.gguf"
llm = Llama(model_path=model_path, n_ctx=2048)

# Query processing
def retrieve_relevant_chunks(query):
    query_embedding = embedder.encode([query])[0].tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=3)
    return results["documents"][0] if "documents" in results else []

# Store and retrieve chat history
def store_chat_history(user_query, bot_response):
    chat_text = f"User: {user_query}\nAssistant: {bot_response}"
    chat_embedding = embedder.encode([chat_text])[0].tolist()
    chat_docs = chat_history_collection.peek(1)  # Retrieve at least one stored document
    chat_count = len(chat_docs["documents"]) if chat_docs and "documents" in chat_docs else 0
    chat_history_collection.add(ids=[f"chat_{chat_count}"], documents=[chat_text], embeddings=[chat_embedding])

def retrieve_chat_history():
    results = chat_history_collection.peek(5)  # Retrieve last 5 chat interactions
    return "\n".join([doc for doc in results["documents"]]) if "documents" in results else ""

# Generate response using GGUF model
def generate_response(query, context=""):
    chat_history = retrieve_chat_history()
    prompt = f"{chat_history}\nContext: {context}\nUser: {query}\nAssistant:"
    response = llm(prompt, max_tokens=256)
    return response["choices"][0]["text"].strip()

# Chatbot interaction in terminal
def chatbot():
    print("\n🔹 Stroke Information Chatbot 🔹")
    print("Ask me anything about strokes. Type 'exit' to quit.\n")

    while True:
        user_query = input("You: ").strip().lower()
        if user_query == "exit":
            print("Chatbot: Goodbye!")
            break
        else:
            context_chunks = retrieve_relevant_chunks(user_query)
            context_text = " ".join(context_chunks)
            response = generate_response(user_query, context_text)
            store_chat_history(user_query, response)
            print(f"Chatbot: {response}\n")

# Run the chatbot
chatbot()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

New PDF successfully processed and stored in ChromaDB!


print_info: n_embd_head_v    = 128
print_info: n_gqa            = 1
print_info: n_embd_k_gqa     = 4096
print_info: n_embd_v_gqa     = 4096
print_info: f_norm_eps       = 0.0e+00
print_info: f_norm_rms_eps   = 1.0e-06
print_info: f_clamp_kqv      = 0.0e+00
print_info: f_max_alibi_bias = 0.0e+00
print_info: f_logit_scale    = 0.0e+00
print_info: f_attn_scale     = 0.0e+00
print_info: n_ff             = 11008
print_info: n_expert         = 0
print_info: n_expert_used    = 0
print_info: causal attn      = 1
print_info: pooling type     = 0
print_info: rope type        = 0
print_info: rope scaling     = linear
print_info: freq_base_train  = 10000.0
print_info: freq_scale_train = 1
print_info: n_ctx_orig_yarn  = 4096
print_info: rope_finetuned   = unknown
print_info: ssm_d_conv       = 0
print_info: ssm_d_inner      = 0
print_info: ssm_d_state      = 0
print_info: ssm_dt_rank      = 0
print_info: ssm_dt_b_c_rms   = 0
print_info: model type       = 7B
print_info: model params     = 6.74 B
pr

...

Chatbot: Hello! A stroke is a medical emergency that occurs when a blood vessel in the brain gets blocked or ruptures, causing brain cells to die. It can affect various functions of the brain, such as movement, feeling, thinking, and behavior. If you suspect someone is having a stroke, it's crucial to act F.A.S.T. and seek medical attention immediately. The faster treatment is received, the better the outcome can be. Remember, a stroke is a medical emergency that requires prompt action!
User: tell me the stroke facts
Assistant: Of course! Here are some key stroke facts:
1. A stroke occurs every 40 seconds, claims the American Heart Association.
2. Strokes are the fifth leading cause of death in the United States, according to the Centers for Disease Control and Prevention (CDC).
3. A stroke can happen to anyone, regardless of age. However, the risk increases with age, with most strokes occurring in people over the age of 65.
4. There are two main types of strokes: ischemic (caused by a

Llama.generate: 287 prefix-match hit, remaining 220 prompt tokens to eval
llama_perf_context_print:        load time =  212553.19 ms
llama_perf_context_print: prompt eval time =   89339.68 ms /   220 tokens (  406.09 ms per token,     2.46 tokens per second)
llama_perf_context_print:        eval time =   90404.23 ms /   123 runs   (  734.99 ms per token,     1.36 tokens per second)
llama_perf_context_print:       total time =  179822.49 ms /   343 tokens


Chatbot: Stroke is a relatively common medical condition, with more than 800,000 people in the United States experiencing a stroke every year. This means that about 1 in 25 people will have a stroke in their lifetime. Additionally, stroke is the 5th leading cause of death in the US, claiming the lives of around 185,000 people each year. It's important to be aware of the signs of a stroke and to act F.A.S.T. in emergency situations to minimize the damage caused by a stroke.

You: tell me the types of a stroke


Llama.generate: 287 prefix-match hit, remaining 289 prompt tokens to eval
llama_perf_context_print:        load time =  212553.19 ms
llama_perf_context_print: prompt eval time =  118513.54 ms /   289 tokens (  410.08 ms per token,     2.44 tokens per second)
llama_perf_context_print:        eval time =  189622.68 ms /   255 runs   (  743.62 ms per token,     1.34 tokens per second)
llama_perf_context_print:       total time =  308337.31 ms /   544 tokens


Chatbot: Of course! There are two main types of strokes:

Ischemic Stroke: This type of stroke occurs when a blood clot blocks a vessel supplying blood to the brain.

Hemorrhagic Stroke: This type of stroke happens when a blood vessel bursts (ruptures) in the brain.


Cerebral Hemorrhage: This type of stroke occurs when there is bleeding in the brain, which can be caused by a ruptured blood vessel or other conditions.


Ischemic Stroke Accounts for about 87% of Strokes


Hemorrhagic Stroke is Less Common but Associated with a Higher Risk of Death

You: exit
Chatbot: Goodbye!
